이전 강의에서 모델 학습했던 코드를 다시 실행해서 학습된 모델을 만들어보겠습니다.

In [ ]:
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import TensorDataset, DataLoader

# 2. 데이터셋 준비 (3개의 Feature를 가진 데이터)
x = np.random.rand(100, 3)
# 간단한 목표 변수 생성: y = 2*x1 - 3*x2 + 5*x3 + 1 + noise
y = 2*x[:, 0] - 3*x[:, 1] + 5*x[:, 2] + 1 + np.random.randn(100)

# PyTorch 텐서로 변환
x_tensor = torch.from_numpy(x).float()
y_tensor = torch.from_numpy(y).float()

# Dataset과 DataLoader 준비
dataset = TensorDataset(x_tensor, y_tensor)
dataloader = DataLoader(dataset, batch_size=10, shuffle=True)  # 데이터를 섞는 옵션을 지정

# 3. 모델 정의 (Sequential 사용)
model = nn.Sequential(
    nn.Linear(3, 5),  # 입력 차원 3, 중간 층 차원 5
    nn.ReLU(),   # 비선형 특성 부여
    nn.Linear(5, 4),  # 중간 층 차원 5, 또 다른 중간 층 차원 4
    nn.ReLU(),
    nn.Linear(4, 1)   # 마지막 층 차원 4, 출력 차원 1
)

# 4. 손실 함수와 옵티마이저 설정
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# 5. 학습
epochs = 200  # 학습을 위해 에포크 수를 늘림
for epoch in range(epochs):
    for inputs, targets in dataloader:
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        optimizer.zero_grad()  # gradient 0으로 초기화
        loss.backward()   # gradient 계산
        optimizer.step()   # weight 수정

# 학습된 모델의 가중치 중 일부 확인
list(model.parameters())[:2]  # 첫 번째 선형 층의 가중치와 편향만 출력


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([10, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


[Parameter containing:
 tensor([[-0.4567, -0.5604, -0.4118],
         [ 0.2682, -0.4997,  0.0448],
         [ 0.3193, -0.3259, -0.2429],
         [-0.5441, -0.3319, -0.4362],
         [ 0.1056,  0.3903,  0.0975]], requires_grad=True),
 Parameter containing:
 tensor([ 0.3382,  0.3751, -0.0424, -0.5101,  0.0540], requires_grad=True)]

이제 학습된 모델에 새로운 데이터를 넣고 모델의 예측값을 구해보겠습니다.  

In [ ]:
# 새로운 데이터 생성 (3개의 특성을 가진 데이터)
new_x = np.random.rand(5, 3).astype(np.float32)  # 예제로 5개의 새로운 데이터 포인트 생성
new_x_tensor = torch.from_numpy(new_x)

print(new_x)

# 모델을 평가 모드로 설정 (필요한 경우, 예를 들어 Dropout이나 BatchNorm과 같은 특정 층의 동작을 평가 모드로 설정)
model.eval()

# 새로운 데이터에 대한 예측 수행
with torch.no_grad():  # 기울기 계산을 비활성화하여 메모리 사용량을 줄이고 계산 속도를 높임
    predictions = model(new_x_tensor)

# 예측 결과 출력
predictions


[[0.09780399 0.9475629  0.30694857]
 [0.91004443 0.34280536 0.18660139]
 [0.3295056  0.93059796 0.24628526]
 [0.9804526  0.39894387 0.29355928]
 [0.6420869  0.66092396 0.94132334]]


tensor([[3.2750],
        [3.3213],
        [3.2761],
        [3.3213],
        [3.3000]])

Train과 eval을 반복하는 경우에는 model.eval() <--> model.train() 모드로 학습과 평가 모드 변환을 해줍니다

* torch.no_grad() : gradient 계산을 안 함으로 리소스 효율 향상, Inference의 경우 gradient 계산 필요 없으므로 주로 사용. with 문법과 함께 사용

* torch.eval(), torch.train(): dropout, barchnorm등 train과 evaluation 단계에서 다른 연산을 하는 경우 전환

* optimizer.zero_grad(): gradient 초기화